아이템 기반 추천은 생각만큼 정확한 결과를 기대하기 힘들다.  
개선 방법으로 개인의 취향 기준으로 추천하는 것으로 협업 필터링(Collaborative Filtering : CF) 알고리즘이다.

### 3.1 협업 필터링의 원리

추천의 대상이 되는 사람과 취향이 비슷한 사람들을  찾아서 공통적으로 좋아하는 제품 서비스를 추천해준다는 아이디어

협업 필터링의 과정
1. 취향이 비슷한 사용자를 유사성을 계산하여 neighbor 그룹으로 분류한다.
2. 분류한 그룹의 인원이 가장 좋게 평가한 아이템의 평점 평균을 내어 값이 높은 아이템을 추천한다.

### 3.2 유사도지표

상관계수
+ 평가 자료가 연속값인 경우 이해하기 쉬운 유사도

코사인 유사도
+ 상관계수는 이해하기 쉽지만 늘 좋은 결과를 가져오지 못한다.
+ 협업 필터링에서 널리 쓰이는 유사도지표로 코사인 유사도가 있다.
+ 아이템을 하나의 차원으로 보고 평가값을 좌표로 하여 각 사용자의 평가값을 벡터로 해서 두 사용자 간의 벡터의 각도를 구할 수 있다.
+ 두 사용자의 평가값이 유사할수록 코사인 값이 크다는 것을 알 수 있다.

타니모토 계수
+ 데이터가 이진값이면 상관계수나 코사인 유사도를 사용할 수 없다.
+ 이 경우 타니모토 계수를 사용한다.
+ 이진수 데이터에 대해 협업 필터링에서 좋은 결과를 보여준다.
+ 타니모토 계수를 변형하여 자카드 계수로 사용하기도 한다.

### 3.3 기본 CF 알고리즘

이웃(neighbor)을 전체 사용자로 하여 모든 사용자의 평점을 가지고 예측한다.
1. 모든 사용자 간의 평가의 유사도를 계산한다.(상관계수, 코사인 유사도 등)
2. 추천 대상이 되는 사람과 다른 사용자의 유사도를 추출한다.
3. 현재 사용자가 평가하지 않은 모든 아이템에 대해 현재 사용자의 예상 평가값을 구한다.  
예상 평가값은 다른 사용자의 해당 아이템에 대한 평가를 현재 사용자와 그 사용자와의 유사도로 가중해서 평균을 낸다.
4. 아이템 중에서 예상 평가값이 가장 높은 N개의 아이템을 추천한다.

2장에서 사용한 코드를 가져와서 실행하고 그 이후부터 살펴본다.

In [9]:
import numpy as np
import pandas as pd

# 데이터 읽어 오기 
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('./data/u.user', sep='|', names=u_cols, encoding='latin-1')
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
          'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']
movies = pd.read_csv('./data/u.item', sep='|', names=i_cols, encoding='latin-1')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('./data/u.data', sep='\t', names=r_cols, encoding='latin-1')

# timestamp 제거 
ratings = ratings.drop('timestamp', axis=1)
# movie ID와 title 빼고 다른 데이터 제거
movies = movies[['movie_id', 'title']]

# train, test 데이터 분리
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

# 정확도(RMSE)를 계산하는 함수 
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# 모델별 RMSE를 계산하는 함수 
def score(model):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

# train 데이터로 Full matrix 구하기 
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')